In [122]:
import pandas as pd
import numpy as np

In [123]:
contaminantes_2016 = pd.read_csv("../Datos/raw/contaminantes/contaminantes_2016.csv")
contaminantes_2017 = pd.read_csv("../Datos/raw/contaminantes/contaminantes_2017.csv")
contaminantes_2018 = pd.read_csv("../Datos/raw/contaminantes/contaminantes_2018.csv")
contaminantes_2019 = pd.read_csv("../Datos/raw/contaminantes/contaminantes_2019.csv")
contaminantes_2020 = pd.read_csv("../Datos/raw/contaminantes/contaminantes_2020.csv")
contaminantes_2021 = pd.read_csv("../Datos/raw/contaminantes/contaminantes_2021.csv")
contaminantes_2022 = pd.read_csv("../Datos/raw/contaminantes/contaminantes_2022.csv")
contaminantes_2023 = pd.read_csv("../Datos/raw/contaminantes/contaminantes_2023.csv")
contaminantes_2024 = pd.read_csv("../Datos/raw/contaminantes/contaminantes_2024.csv")

meteorologia_2016 = pd.read_csv("../Datos/raw/meteorologia/meteorologia_2016.csv")
meteorologia_2017 = pd.read_csv("../Datos/raw/meteorologia/meteorologia_2017.csv", header=10)
meteorologia_2018 = pd.read_csv("../Datos/raw/meteorologia/meteorologia_2018.csv", header=10)
meteorologia_2019 = pd.read_csv("../Datos/raw/meteorologia/meteorologia_2019.csv", header=10)
meteorologia_2020 = pd.read_csv("../Datos/raw/meteorologia/meteorologia_2020.csv", header=10)
meteorologia_2021 = pd.read_csv("../Datos/raw/meteorologia/meteorologia_2021.csv")
meteorologia_2022 = pd.read_csv("../Datos/raw/meteorologia/meteorologia_2022.csv", header=10)
meteorologia_2023 = pd.read_csv("../Datos/raw/meteorologia/meteorologia_2023.csv", header=10)
meteorologia_2024 = pd.read_csv("../Datos/raw/meteorologia/meteorologia_2024.csv")

In [124]:
def impute_nan_dates(date_series):
    """
    Impute NaN values in a pandas Series of date strings.
    If the previous value ends with '23:00', impute with same date but '24:00'.
    If the previous value ends with '24:00', copy it to the current NaN.
    Returns a new pandas Series with imputed values.
    """
    dates = date_series.copy()
    for i in range(len(date_series)):
        if isinstance(date_series[i], float):
            prev = dates[i-1]
            if prev.endswith('23:00:00'):
                date_part = prev.rsplit(' ', 1)[0]
                dates[i] = date_part + " 24:00:00"
            elif prev.endswith('24:00:00'):
                dates[i] = prev
    return dates

In [125]:
contaminantes_2022["date"] = impute_nan_dates(contaminantes_2022["date"])

In [126]:
from datetime import datetime, timedelta

def decrease_hour(date_val):
    # Handle datetime-like objects
    if isinstance(date_val, pd.Timestamp):
        return (date_val - timedelta(hours=1)).strftime('%d/%m/%Y %H:%M')
    
    if isinstance(date_val, str):
        # Try parsing ISO format first
        try:
            dt = datetime.fromisoformat(date_val)
            return (dt - timedelta(hours=1)).strftime('%d/%m/%Y %H:%M')
        except ValueError:
            pass  # Not ISO format, try custom logic
        
        try:
            date_part, time_part = date_val.strip().split()
            day, month, year = map(int, date_part.split('/'))
            hour, minute = map(int, time_part.split(':'))

            if hour == 24:
                hour = 23
            elif hour == 0:
                dt = datetime(day=day, month=month, year=year) - timedelta(days=1)
                return dt.strftime('%d/%m/%Y') + ' 23:00'
            else:
                hour -= 1

            return f'{date_part} {hour:02}:{minute:02}'
        except Exception as e:
            return None  # Or log the error and original value

    return None  # If value is neither str nor datetime

In [136]:
dataframes = [contaminantes_2016, contaminantes_2017, contaminantes_2018, contaminantes_2019,
              contaminantes_2020, contaminantes_2021, contaminantes_2022, contaminantes_2023,
              contaminantes_2024]
df = pd.concat(dataframes)

df['date'] = df['date'].apply(decrease_hour)

df['date'] = pd.to_datetime(df['date'])
#df = df.dropna(subset=['date'])

In [137]:
df = df.groupby(["date","id_parameter"])["value"].mean().reset_index()

In [138]:
pivoted_df = df.pivot(index='date', columns='id_parameter', values='value')
pivoted_df = pivoted_df.reset_index()

In [139]:
pivoted_df

id_parameter,date,CO,NO,NO2,NOX,O3,PM10,PM2.5,PMCO,SO2
0,2016-01-01 00:00:00,1.338889,16.052632,46.210526,62.263158,9.739130,152.000000,81.785714,36.111111,18.294118
1,2016-01-01 01:00:00,1.494444,18.473684,46.315789,64.842105,7.739130,175.631579,111.857143,44.333333,21.176471
2,2016-01-01 02:00:00,1.444444,24.368421,43.000000,67.368421,6.652174,178.473684,127.928571,46.222222,31.647059
3,2016-01-01 03:00:00,1.383333,26.684211,41.000000,67.842105,6.000000,166.894737,133.500000,41.444444,39.235294
4,2016-01-01 04:00:00,1.383333,32.000000,39.315789,71.473684,4.434783,168.842105,135.285714,36.333333,41.176471
...,...,...,...,...,...,...,...,...,...,...
78533,2024-12-31 18:00:00,0.901667,3.650000,47.142857,52.500000,51.240000,73.125000,34.384615,24.857143,3.428571
78534,2024-12-31 19:00:00,0.650833,3.100000,35.666667,39.950000,41.840000,78.812500,26.615385,34.571429,2.904762
78535,2024-12-31 20:00:00,0.502500,2.700000,30.809524,33.900000,32.280000,59.933333,21.500000,32.833333,1.904762
78536,2024-12-31 21:00:00,0.554583,3.850000,32.666667,37.150000,24.000000,62.266667,23.083333,25.166667,1.857143


In [91]:
for i in range(len(contaminantes_2022["date"])):
    if isinstance(contaminantes_2022["date"][i], float):
        print(contaminantes_2022["date"][i-1:i+2])


In [92]:
contaminantes_2022["date"]

0          2022-01-01 01:00:00
1          2022-01-01 01:00:00
2          2022-01-01 01:00:00
3          2022-01-01 01:00:00
4          2022-01-01 01:00:00
                  ...         
2426238    2022-12-31 23:00:00
2426239    2022-12-31 23:00:00
2426240    2022-12-31 23:00:00
2426241    2022-12-31 23:00:00
2426242    2022-12-31 23:00:00
Name: date, Length: 2426243, dtype: object